# Code Setup

This is the first file I created. It converts the Rocket League replay files to pandas dataframes. 

In [1]:
import carball
from carball.json_parser.game import Game
from carball.analysis.analysis_manager import AnalysisManager

The following function renames the columns by team color instead of a multiindex with the player names

In [2]:
def col_names(df):
    blue_name = df.columns[1][0]
    orange_name = df.columns[-1][0]
    
    # if blue y position is above 0, we have blue and orange backwards
    if df.loc[df.index == 1, (blue_name, "pos_y")][1] > 0:
        blue_name, orange_name = orange_name, blue_name
    
    a = []
    for i in df.columns:
        if i[0] == blue_name:
            a.append("blue_" + i[1])
        elif i[0] == orange_name:
            a.append("orange_" + i[1])
        else:
            a.append("_".join(i))

    df.columns = a
    return df

This function converts the replay file into a dataframe. 

In [3]:
def prepare(file):
    _json = carball.decompile_replay(file)
    game = Game()
    game.initialize(loaded_json=_json)
    
    analysis_manager = AnalysisManager(game)
    analysis_manager.create_analysis()
    
    dataframe = analysis_manager.get_data_frame()
    dataframe = col_names(dataframe)
    
    if dataframe.shape[1] != 66:
        dataframe.insert(61, 'game_is_overtime', 'no_OT')
    
    dataframe.insert(0, "ID", dataframe.index)
    
    # the parsing sometimes marks 0 seconds as null so we need to fix that
    dataframe.loc[dataframe["game_seconds_remaining"].isnull(), "game_seconds_remaining"] = 0
    
    return dataframe

This function finds the next goal scoring team for any given row, and the difference in time between the two goals

In [4]:
def next_goal_regulation(row, df):
    try:
        # find frame before next goal
        ng = df.loc[(df["game_goal_number"].isnull()) & (df["ID"] > row), "ID"].iloc[0] - 1
        
        # depending on the ball location, we can tell who scored the next goal
        if df.loc[ng, "ball_pos_y"] > 0:
            return (0, df.loc[row, "game_seconds_remaining"] - df.loc[ng, "game_seconds_remaining"])
        else:
            return (1, df.loc[row, "game_seconds_remaining"] - df.loc[ng, "game_seconds_remaining"])
    except:
        return (None, None)


def next_goal_ot(df):
    # check where the ball is to end overtime
    if df.iloc[-1, df.columns.get_loc("ball_pos_y")] > 0:
        # if ball is on orange half, team 0 (blue) scored in ot
        return 0
    else:
        # else team 1 (orange) scored
        return 1


def next_goal(df):
    if df.loc[1, "game_is_overtime"] == "no_OT":
        # if game doesn't go to overtime, we just need to run the regulation function
        temp = df["ID"].apply(lambda x: next_goal_regulation(x, df))
        df["game_next_goal"] = [x[0] for x in temp]
        df["time_to_next_goal"] = [x[1] for x in temp]
    else:
        # if game goes to overtime, split df into ot and regulation
        df_ot = df.loc[df["game_is_overtime"].isnull() == False, ]
        df = df.loc[df["game_is_overtime"].isnull(), ]
        
        # run ot function on the ot dataframe
        df_ot.loc[:, "game_next_goal"] = next_goal_ot(df_ot)
        
        # get times difference of the OT goal
        df_ot["time_to_next_goal"] = df_ot.iloc[-1, df.columns.get_loc("game_seconds_remaining")] - df_ot["game_seconds_remaining"]
        
        # run regulation function on the regulation dataframe
        temp = df["ID"].apply(lambda x: next_goal_regulation(x, df))
        df["game_next_goal"] = [x[0] for x in temp]
        df["time_to_next_goal"] = [x[1] for x in temp]
        
        # bind the dataframe back together
        df = df.append(df_ot)
    
    # remove all rows after goals
    df = df.loc[df["game_goal_number"].isnull() == False, ]
    
    return df

From this point forward, I basically just looped through my directory for all of the replays. I did it in groups of 20 so I could work on it periodically. It took about 6 minutes for every 20 replays. 

In [5]:
import os

In [6]:
def data_setup(string, directory):
    i = 0
    for file in os.listdir(directory):
        print(os.fsdecode(file))
        df = prepare(string + os.fsdecode(file))
        df = next_goal(df)
        if i == 0:
            df_total = df
            i += 1
        else:
            df_total = df_total.append(df)
    return df_total

This is just practice to make sure my code works

In [7]:
string = "./Replays/practice/"
directory = os.fsencode(string)
df_total = data_setup(string, directory)
df_total.shape

5c402249-2714-47a3-bd98-48d7e631e95d.replay


Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 362 by JJ.
Goal is not shot: frame 6324 by JJ.
The player never hit the ball during the "carry"


74854701-b08b-416f-b849-1152eeb5abea.replay


Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 3892 by WAJEH
Goal is not shot: frame 6492 by Iryzer


924303e9-4a99-4ef8-9c15-13484b7bf72f.replay


Goal is not shot: frame 5292 by Dider
Goal is not shot: frame 9079 by fivra
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"
C:\Users\mhugh\anaconda3\envs\carball\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\mhugh\anaconda3\envs\carball\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\mhugh\A

b4b9e582-214f-4372-8d24-95e2a2f3e8ac.replay


Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 5278 by shad
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"


(38820, 69)

Here are the replays I used for my experiment

In [8]:
import time

In [9]:
start_time = time.time()
string = "./Replays2/test/"
directory = os.fsencode(string)
df_total = data_setup(string, directory)
print(time.time() - start_time)
df_total.shape

000dcd74-b0db-4527-868a-d5b6adb299d2.replay


Dropping these columns[('game', 'is_overtime')]
Could not find hit for goal: frame_number: 250
player_id {
  id: "76561198336321389"
}

Goal is not shot: frame 1100 by itsok.
Goal is not shot: frame 7814 by MainDavid101
The player never hit the ball during the "carry"


022b136a-fbbd-4f44-885e-6aa41db9b72b.replay


Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 1556 by Duckz
Goal is not shot: frame 5924 by Duckz
Goal is not shot: frame 9278 by Duckz


1de63155-a40d-4f1d-bcac-1457e007735b.replay


...

Found bot not in bot list
Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 1156 by Taha on Twitch
The player never hit the ball during the "carry"


fbcb688d-7141-47fa-9623-9b0394a98e11.replay


Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 4267 by insta-nssbali
Goal is not shot: frame 9481 by Hyderr
The player never hit the ball during the "carry"


246.8154752254486


(177101, 69)

In [10]:
df_total.to_pickle("testing_data_v3.pkl")

In [11]:
start_time = time.time()
string = "./Replays2/train1/"
directory = os.fsencode(string)
df_total = data_setup(string, directory)
print(time.time() - start_time)
df_total.shape

1be6251e-11cd-49a0-b4bc-5846f84cb6b9.replay


Dropping these columns[('game', 'is_overtime')]


24385949-14b1-49cc-a8b1-a55fae5932cd.replay


Dropping these columns[('game', 'is_overtime')]
The player never hit the ball during the "carry"


2f04c11a-1ef5-4ba2-9470-ce0d0ff1130b.replay


...

Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 8366 by makus
The player never hit the ball during the "carry"


f6f19dba-ab5e-4b30-85a8-3811ac1f65f8.replay


Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 1507 by slumplight ッ
Goal is not shot: frame 2680 by slumplight ッ
Goal is not shot: frame 6716 by slumplight ッ
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"


232.44508981704712


(168558, 69)

In [12]:
df_total.to_pickle("training_data1_v3.pkl")

In [13]:
start_time = time.time()
string = "./Replays2/train2/"
directory = os.fsencode(string)
df_total = data_setup(string, directory)
print(time.time() - start_time)
df_total.shape

1d163c8b-2421-46d5-9997-84d1993ccb74.replay


Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 1291 by Geo
The player never hit the ball during the "carry"


232d52cb-0b48-4316-a4eb-41f4f082c07a.replay


Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 7133 by ban.
Goal is not shot: frame 10159 by .


3efb8106-b01d-44f4-894a-440ecb2d4215.replay


...

Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 6986 by Tryzin pushing P
Goal is not shot: frame 10338 by Tryzin pushing P
The player never hit the ball during the "carry"


fa8dcda0-86b0-449e-9e36-0b6fe9ad7152.replay


Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 2607 by Dimes
Goal is not shot: frame 6221 by Dimes
The player never hit the ball during the "carry"


231.07231783866882


(168337, 69)

In [14]:
df_total.to_pickle("training_data2_v3.pkl")

In [15]:
start_time = time.time()
string = "./Replays2/train3/"
directory = os.fsencode(string)
df_total = data_setup(string, directory)
print(time.time() - start_time)
df_total.shape

00e23f58-d6cb-46e1-b905-da75be060e9c.replay


Goal is not shot: frame 2575 by :)
Goal is not shot: frame 4437 by :)
C:\Users\mhugh\anaconda3\envs\carball\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\mhugh\anaconda3\envs\carball\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\mhugh\AppData\Local\Temp/ipykernel_5972/324666517.py:40: SettingWithCopyWarning: 
A value is trying to be set o

1e94df5c-5a30-4069-aa5c-c65b17e1b2e5.replay


Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 1178 by ★Toxiic★
Goal is not shot: frame 4193 by trace
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"
C:\Users\mhugh\anaconda3\envs\carball\lib\site-packages\numpy\lib\format.py:431: UserWarning: Stored array in format 3.0. It can only be read by NumPy >= 1.17
  header = _wrap_header_guess_version(header)


34470e3a-fad0-4d0b-b218-19d473de1fbe.replay


...

Found bot not in bot list
Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 3088 by grandhx


bd8ce728-eeb4-4831-9649-8ffac729c0a1.replay


Dropping these columns[('game', 'is_overtime')]


234.83639430999756


(173663, 69)

In [16]:
df_total.to_pickle("training_data3_v3.pkl")

In [17]:
start_time = time.time()
string = "./Replays2/accuracy/"
directory = os.fsencode(string)
df_total = data_setup(string, directory)
print(time.time() - start_time)
df_total.shape

02511bd4-8330-40e4-a500-d4db03623feb.replay


Found bot not in bot list
Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 9092 by cam.
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"
C:\Users\mhugh\anaconda3\envs\carball\lib\site-packages\numpy\lib\format.py:431: UserWarning: Stored array in format 3.0. It can only be read by NumPy >= 1.17
  header = _wrap_header_guess_version(header)


0c40e95c-b00b-4c95-b7e4-b24115b035f6.replay


Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 355 by sky.
The player never hit the ball during the "carry"


15b4d253-0da0-4101-b678-f78ec61a4e3c.replay


...

Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 3525 by .Wahvey
The player never hit the ball during the "carry"


c7274875-1658-4b0d-86a7-949703a24982.replay


Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 2471 by Davoof
Goal is not shot: frame 8878 by Davoof
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"


237.55948901176453


(177947, 69)

In [18]:
df_total.to_pickle("accuracy_data_v3.pkl")

In [9]:
start_time = time.time()
string = "./Replays2/final_analysis/"
directory = os.fsencode(string)
df_total = data_setup(string, directory)
print(time.time() - start_time)
df_total.shape

EEBBEFDB4F55F6E19D8DAA962AC39053.replay


Player Fusion Admin as player has no MatchScore.
Score is not found for player
Ignoring player: Fusion Admin as player has no team.
Player JohnnyBoi_i as player has no MatchScore.
Score is not found for player
Ignoring player: JohnnyBoi_i as player has no team.
Player Shogun as player has no MatchScore.
Score is not found for player
Ignoring player: Shogun as player has no team.
Goal is not shot: frame 1204 by Firstkiller
The player never hit the ball during the "carry"
C:\Users\mhugh\anaconda3\envs\carball\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\mhugh\anaconda3\envs\carball\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 

15.022202968597412


C:\Users\mhugh\AppData\Local\Temp/ipykernel_18640/324666517.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["game_next_goal"] = [x[0] for x in temp]
C:\Users\mhugh\AppData\Local\Temp/ipykernel_18640/324666517.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["time_to_next_goal"] = [x[1] for x in temp]


(11522, 69)

In [10]:
df_total.to_pickle("analysis_data_v3.pkl")